In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql
import pandas as pd
import numpy as np

In [5]:
import pandas as pd

# Read the CSV
netflix_titles = pd.read_csv('mycsvfile.csv')

# Safely convert 'date_added' to datetime format
netflix_titles['date_added'] = pd.to_datetime(netflix_titles['date_added'], errors='coerce')

# Now format the date column (after ensuring it's datetime)
netflix_titles['date_added'] = netflix_titles['date_added'].dt.strftime('%Y-%m-%d')

# Extract required values
fullList = netflix_titles['title'].dropna().tolist()
genreList = list(set(netflix_titles['listed_in'].dropna().tolist()))


In [7]:
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019,4.1,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09-09,2016,5.2,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2013,8.2,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2016,6.4,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017,5.7,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [9]:
showDim = netflix_titles.drop(['date_added', 'release_year', 'rating', 'listed_in'], axis=1) # delete the columns that are not needed
showDim = showDim.where((pd.notnull(showDim)), None) # convert empty values to "None" values
showDimList = []
for row in showDim.values.tolist():
    showDimList.append(tuple(row)) # format data into a list of tuples before inserting to database

# showDimList: show data for show_dim table

In [9]:
netflix_origionals = pd.read_csv('netflix_originals.csv') # using pandas to read the csv file
netflix_origionals = netflix_origionals[['Title','Seasons','Length','Netflix Exclusive Regions','Status']] # select the columns to keep
netflix_origionals = netflix_origionals.where((pd.notnull(netflix_origionals)), None) # convert empty values to "None" values
netflix_origionals.drop_duplicates(subset ="Title",
                     keep = 'first', inplace = True) # delete duplicate values
netflix_origionalsList = [tuple(l) for l in netflix_origionals.values.tolist()] # format data into a list of tuples before inserting to database
#print(netflix_origionalsList) #: data for original_dim table

In [10]:
netflix_origionals.head()

,Title,Seasons,Length,Netflix Exclusive Regions,Status
0,House of Cards,"6 seasons, 73 episodes",42–59 min.,Worldwide,Ended
1,Hemlock Grove,"3 seasons, 33 episodes",45–58 min.,Worldwide,Ended
2,Orange Is the New Black,"7 seasons, 91 episodes",50–92 min.,Worldwide,Ended
3,Marco Polo,"2 seasons, 20 episodes",48–65 min.,Worldwide,Ended
4,Bloodline,"3 seasons, 33 episodes",48–68 min.,Worldwide,Ended


In [12]:
netflix_stocks = pd.read_csv('NFLX.csv', parse_dates=['Date']) # using pandas to read the csv file

In [13]:
netflix_stocks.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
1,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
3,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
4,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


In [15]:
dateDim = pd.DataFrame({'date': pd.date_range(start='2002-05-23', end='2020-08-03')}) # create a dataframe that has dates ranging from 2002-05-23 to 2020-08-03
dateDim['date_id'] = dateDim.index + 1 # create date_id column and assign id numbers starting from 1
dateDim['date'] = dateDim['date'].dt.strftime("%Y-%m-%d") # format date column to YYYY-MM-DD
dateDim['year'] = pd.DatetimeIndex(dateDim['date']).year # using the year information from date colunn to create year column
dateDim = dateDim.reindex(columns=['date_id','date','year']) # re-arrange the order of columns
#dateDim.date = pd.to_datetime(dateDim.date)
dateDim.date_id = dateDim.date_id.astype(str) # convert data in date column to string

In [16]:
dateDimList = [tuple(l) for l in dateDim.values.tolist()] # format data into a list of tuples before inserting to database
# dateDimList: data for date_dim table

In [17]:
dateDim.date = pd.to_datetime(dateDim.date) # convert data in date column to datatype date

factsStockDF = pd.merge(netflix_stocks, dateDim, left_on='Date', right_on='date', how='inner') # inner join dataframes netflix_stocks and dataDim on date
factsStockDF = factsStockDF.drop(['Date', 'date', 'year'], axis=1) # delete columns that are not needed
factsStockDF = factsStockDF.reindex(columns=['date_id','Open','High','Low','Close','Adj Close', 'Volume']) # re-arrange the order of columns
# factsStockDF['Date'] = factsStockDF['Date'].dt.strftime("%Y-%m-%d")


In [18]:
factsStockList = [tuple(l) for l in factsStockDF.values.tolist()] # format data into a list of tuples before inserting to database
# factsStockList: data for facts_stock_prices table

In [19]:
factsRating = pd.merge(netflix_titles, netflix_origionals, left_on='title', right_on='Title', how='left') # left join dataframes netflix_titles and netflix_originals on titles
factsRating.date_added = pd.to_datetime(factsRating.date_added) # convert data in date_added column to datatype date

In [20]:
factsRatingDF = pd.merge(factsRating, dateDim, left_on='date_added', right_on='date', how='left') # left join dataframes factsRating and dateDim on date_added/date

In [21]:
factsRatingDF = factsRatingDF[['show_id','title','Title','date_id','listed_in','rating']] # select only columns needed

In [22]:
factsRatingDF = factsRatingDF.where((pd.notnull(factsRatingDF)), None) # convert empty values to "None" values

In [25]:
factsRatingList = [tuple(l) for l in factsRatingDF.values.tolist()] # format data into a list of tuples before inserting to database
#print(factsRatingList) # data for facts_IMDB_rating table

In [ ]:
# create show_dim table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.curs``or()

# cursor.execute("DROP TABLE IF EXISTS show_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable1 = """CREATE TABLE show_dim(
                 show_id INT not null,
                 type VARCHAR(255) null,
                 title VARCHAR(255) not null,
                 director VARCHAR(255) null,
                 cast MEDIUMTEXT null,
                 country VARCHAR(255) null,
                 duration VARCHAR(255) null,
                 description MEDIUMTEXT null,
                 PRIMARY KEY (show_id, title));"""

cursor.execute(createTable1)

insertData1 = "INSERT INTO show_dim VALUES (%s,%s,%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData1, showDimList)
db.commit()

db.close() # close the connection to mysql

In [30]:
# create original_dim table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS original_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable2 = """CREATE TABLE original_dim(
                  original_title VARCHAR(255) not null,
                  seasons VARCHAR(255) null,
                  length VARCHAR (255) null,
                  regions VARCHAR (255) null,
                  status VARCHAR (255) null,
                  PRIMARY KEY (original_title));"""

cursor.execute(createTable2)

insertData2 = "INSERT INTO original_dim VALUES (%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData2, netflix_origionalsList)
db.commit()

db.close() # close the connection to mysql

In [31]:
# create date_dim table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS date_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable3 = """CREATE TABLE date_dim(
                  date_id VARCHAR(50) not null,
                  date DATE null,
                  year INT null,
                  PRIMARY KEY (date_id));"""

cursor.execute(createTable3)

insertData3 = "INSERT INTO date_dim VALUES (%s,%s,%s);" # insert data into table
cursor.executemany(insertData3, dateDimList)
db.commit()

db.close() # close the connection to mysql

In [33]:
# create genre_dim table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS genre_dim;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable4 = """CREATE TABLE genre_dim(
                  listed_in VARCHAR(255) not null,
                  PRIMARY KEY (listed_in));"""

cursor.execute(createTable4)

insertData4 = "INSERT INTO genre_dim VALUES (%s);" # insert data into table
val = [[item] for item in genreList]
cursor.executemany(insertData4, val)
db.commit()

db.close() # close the connection to mysql

In [34]:
# create facts_imdb_rating table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_imdb_rating;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable5 = """CREATE TABLE facts_imdb_rating(
                  show_id INT not null,
                  title VARCHAR(255) not null,
                  original_title VARCHAR(255) null,
                  date_id VARCHAR(50) null,
                  listed_in VARCHAR(255) null,
                  rating FLOAT null,
                  PRIMARY KEY (show_id),
                  FOREIGN KEY (show_id, title) REFERENCES show_dim(show_id, title),
                  FOREIGN KEY (original_title) REFERENCES original_dim(original_title),
                  FOREIGN KEY (date_id) REFERENCES date_dim(date_id),
                  FOREIGN KEY (listed_in) REFERENCES genre_dim(listed_in)
                  );"""

cursor.execute(createTable5)

insertData5 = "INSERT INTO facts_imdb_rating VALUES (%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData5, factsRatingList)
db.commit()

db.close() # close the connection to mysql

In [35]:
# create facts_stock_prices table and insert data into it
db = pymysql.connect(host="localhost",user="root",password="hassan@2728f",database="netflix_etl") # mysql connection credentials, pass arguments as keywords
cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS facts_stock_prices;") # delete the table if it already exists

# create table with attributes based on the dimensional model
createTable6 = """CREATE TABLE facts_stock_prices(
                  date_id VARCHAR(50) not null,
                  open FLOAT null,
                  high FLOAT null,
                  low FLOAT null,
                  close FLOAT null,
                  adj_close FLOAT null,
                  volume FLOAT null,
                  PRIMARY KEY (date_id),
                  FOREIGN KEY (date_id) REFERENCES date_dim(date_id)
                  );"""

cursor.execute(createTable6)

insertData6 = "INSERT INTO facts_stock_prices VALUES (%s,%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData6, factsStockList)
db.commit()

db.close() # close the connection to mysql

In [36]:
import pymysql

# Establish connection
db = pymysql.connect(
    host="localhost",       
    user="root",                  
    password="hassan@2728f",    
    database="netflix_etl",
    port=3306                   
)

# Create a cursor to execute queries
cursor = db.cursor()

# Test the connection
cursor.execute("SELECT VERSION();")
version = cursor.fetchone()
print("Connected to MySQL Server version:", version[0])

# Always close when done
db.close()


Connected to MySQL Server version: 8.0.41


In [39]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [40]:
import pymysql
from tabulate import tabulate

db = pymysql.connect(host="localhost", user="root", password="hassan@2728f", database="netflix_etl")
cursor = db.cursor()

cursor.execute("SELECT * FROM show_dim LIMIT 5;")
rows = cursor.fetchall()

# Get column names
columns = [desc[0] for desc in cursor.description]

# Print as table
print(tabulate(rows, headers=columns, tablefmt="grid"))

db.close()


+-----------+--------+---------------------------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   show_id | type   | title                           | director          | cast                                                                                                                                                                    | country       | duration   | description                                                                                                                                                                                                         |
+===========+=======

In [ ]:
-